7.1  Create a crawler using the webcrawler provided and discussed in my jupyter notebook ( https://github.com/pschragger/big-data-python-class/blob/master/Lectures/Lecture_7_-_Link_Analysis.ipynb) , have it crawl the first 200 pages from a base URL of your choosing

start project myfirstcrawler
cd basic_crawler
scrapy genspider example example.com

In [ ]:
# %load basic_crawler/basic_crawler/items.py

# Define here the models for your scraped items
#
# See documentation in:
# http://doc.scrapy.org/en/latest/topics/items.html

import scrapy


class BasicCrawlerItem(scrapy.Item):
    # define the fields for your item here like:
    # name = scrapy.Field()
    title = scrapy.Field()
    email = scrapy.Field()
    comments = scrapy.Field()
    form = scrapy.Field()
    location_url = scrapy.Field()


In [ ]:
# %load basic_crawler/basic_crawler/spiders/spiderman.py
from scrapy.spiders import BaseSpider
from scrapy.selector import Selector
from basic_crawler.items import BasicCrawlerItem
from scrapy.http import Request
import re


class MySpider(BaseSpider):
	name = "basic_crawler"
	allowed_domains = ['packtpub.com']
	start_urls = ["https://www.packtpub.com"]

	def parse(self, response):
		hxs = Selector(response)


		#CODE for scraping Forms
		forms = hxs.xpath('//form/@action').extract()
		for form in forms:
			formy = BasicCrawlerItem()
			formy["form"] = form
			formy["location_url"] = response.url
			yield formy

		#CODE for scraping emails
		emails = hxs.xpath("//*[contains(text(),'@')]").extract()
		for email in emails:
			com = BasicCrawlerItem()
			com["email"] = email
			com["location_url"] = response.url
			yield com


		#CODE for scraping comments
		comments = hxs.xpath('//comment()').extract()
		for comment in comments:
			com = BasicCrawlerItem()
			com["comments"] = comment
			com["location_url"] = response.url
			yield com

		visited_links=[]
		links = hxs.xpath('//a/@href').extract()
                link_validator= re.compile("^(?:http|https):\/\/(?:[\w\.\-\+]+:{0,1}[\w\.\-\+]*@)?(?:[a-z0-9\-\.]+)(?::[0-9]+)?(?:\/|\/(?:[\w#!:\.\?\+=&amp;%@!\-\/\(\)]+)|\?(?:[\w#!:\.\?\+=&amp;%@!\-\/\(\)]+))?$")

		for link in links:
			if link_validator.match(link) and not link in visited_links:
				visited_links.append(link)
				yield Request(link, self.parse)
			else:
				full_url=response.urljoin(link)
				visited_links.append(full_url)
				yield Request(full_url, self.parse)


scrapy crawl basic_crawler -o books.csv

In [12]:
import pandas as pd

df = pd.read_csv('basic_crawler/books.csv')
df.head(200)

,title,form,email,comments,location_url
0,NaN,/add_to_cart/31632,NaN,NaN,https://www.packtpub.com
1,NaN,NaN,<style>\n\t\t\t@media only screen and (max-wid...,NaN,https://www.packtpub.com
2,NaN,NaN,<style>\n\t\t\t@media only screen and (max-wid...,NaN,https://www.packtpub.com
3,NaN,NaN,<style>\n\t\t\t@media only screen and (max-wid...,NaN,https://www.packtpub.com
4,NaN,NaN,<style>\n\t\t\t@media only screen and (max-wid...,NaN,https://www.packtpub.com
5,NaN,NaN,NaN,"<!-- <link href=""//fonts.googleapis.com/css?fa...",https://www.packtpub.com
6,NaN,NaN,NaN,"<!--[if lt IE 10]>\n <link href=""/sites...",https://www.packtpub.com
7,NaN,NaN,NaN,<!--Maxymiser script start -->,https://www.packtpub.com
8,NaN,NaN,NaN,<!--Maxymiser script end -->,https://www.packtpub.com
9,NaN,NaN,NaN,<!-- Google Tag Manager -->,https://www.packtpub.com


Reference
https://www.analyticsvidhya.com/blog/2017/07/web-scraping-in-python-using-scrapy/
https://www.lynda.com/Python-tutorials/Creating-crawler-Scrapy/521198/532912-4.html
